In [ ]:
# Creation of a map displaying a random numerical variable on the european countries. 
# The numerical values assigned to each of the countries are randomly generated 
# using numpy.random.uniform and saved into a dictionary having the country code 
# as the key. This dict is transformed to a Pandas DataFrame with 1 row per country and 
# having ‘iso2code’ and ‘value’ as columns. The graduated legend is build using
# the Viridis Plotly colorscale
import numpy as np
import pandas as pd
import plotly.express as px
from ipyleaflet import basemaps
from ipywidgets import widgets, Layout, HTML
from IPython.display import display
from voilalibrary import leafletMap, svgUtils

country_codes = ['LV','AL','CH','AT','HR','ES','IS','RO','IT','VA','HU','SE','NL','CZ','DE','FR',
                 'ME','BE','SI','LU','IE','BA','MC','BG','PL','LI','GB','RS','SM','DK','IM',
                 'EE','SK','GR','LT','NO','PT','AD','MK','MT','GI','FI','XK','CY']

country_selected = ['IT', 'CH']

# Generate random values and create a dictionary: key=countrycode, value=random in [0.0,100.0]
d = dict(zip(country_codes, list(np.random.uniform(size=len(country_codes),low=0.0,high=100.0))))

# Create a pandas dataframe from the dictionary
df = pd.DataFrame(d.items(), columns=['iso2code', 'value'])

m = leafletMap.countriesMap(df,
                            code_column='iso2code',
                            height='550px',
                            stroke_width=1.0,
                            stroke_selected='red',
                            detailedcountries=True,
                            basemap=basemaps.Stamen.Terrain,
                            colorlist=px.colors.sequential.Viridis,
                            codes_selected=country_selected,
                            center=[43,12], zoom=5)


svg = svgUtils.graduatedLegend(df, code_column='iso2code',
                               codes_selected=country_selected, stroke_selected='red', 
                               colorlist=px.colors.sequential.Viridis, stdevnumber=2.0, 
                               legendtitle='Legend title', legendunits='Random numbers',
                               fontsize=16, width=240, height=530)

outlegend = widgets.Output(layout=Layout(width='360px',height='550px'))
with outlegend:
    display(HTML(svg))
    
widgets.HBox([m,outlegend])

In [ ]:
# Creation of a categorycal map with a legend
# The example geojson is used which contains some polygons 
# of the landuse map of the city of Dresden
# The attribute 'fclass' contains the landuse class name
import plotly.express as px
from IPython.display import display
from ipywidgets import widgets, Layout
from voilalibrary import leafletMap, svgUtils, geojsonUtils

# Load landuse example and get unique landuse classes
filepath = './data/landuse.geojson'
geojson = geojsonUtils.geojsonLoadFile(filepath)
landuses = sorted(list(set(geojsonUtils.geojsonAll(geojson,'fclass'))))

# Create a colormap (dictionary that maps landuses to colors)
colors   = px.colors.qualitative.Dark24
colormap = dict(zip(landuses, colors))

m = leafletMap.geojsonCategoricalMap(filepath,
                                     'fclass',
                                     stroke_width=1.0,
                                     stroke='black', 
                                     colormap=colormap,
                                     width='79%',
                                     height='700px',
                                     center=[51.005,13.6],
                                     zoom=12,
                                     basemap=basemaps.CartoDB.Positron,
                                     style={'opacity': 1, 'dashArray': '0', 'fillOpacity': 1})

outlegend = widgets.Output(layout=Layout(width='230px',height='680px'))
with outlegend:
    display(HTML(svgUtils.categoriesLegend("Landuse legend", landuses, colorlist=colors[:len(landuses)])))
widgets.HBox([m,outlegend])

In [ ]:
# Example of display of a vector dataset managed by the collectionsExplorer dashboard
# See: https://jeodpp.jrc.ec.europa.eu/eu/dashboard/voila/render/CollectionsExplorer.ipynb
from jeodpp import inter, imap
from IPython.display import display
from voilalibrary import interMap

# Creation of the interactive map
m = imap.Map()

# Creation of the vector layer for the UTM grid zones
v = inter.Collection(inter.collections.BaseData.GeographicalGridSystems.UTMgrid)

# Setting the legend
v.legendSet('line','stroke','#0000ff')
v.legendSet('line','stroke-width','0.25')
v.legendSingle("#bbffbb")
v.opacity(128)

# Add the vector layer to the map
layer = m.addLayer(v.toLayer())

# Define identify operation at click on a polygon
interMap.CustomIdentifyPopup(m, v)

# Display the map
display(m)

### <b>Creation of a bivariate choropleth legend for a polygon vector layer</b>.
See [Bivariate Choropleth Maps: A How-to Guide](https://www.joshuastevens.net/cartography/make-a-bivariate-choropleth-map/) for the idea. The function creates a legend for vector layer v based on two attributes of the layer and returns a string containing the SVG representation of the legend (that can be displayed using display(HTML(svgstring) call)

In [ ]:
from ipywidgets import widgets, Layout, HTML
from IPython.display import display

from jeodpp import inter, imap
from voilalibrary import interMap, geojsonUtils

# Load data on italian provinces
geojson = geojsonUtils.geojsonLoadFile('./data/ItalyProvinces.geojson')
vector = interMap.interGeojsonToVector(geojson)
vector = vector.parameter("identifyfield", "SIGLA DEN_PROV SHAPE_AREA")
vector = vector.parameter("identifyseparator", "<br>")

# Create and display a Map instance
m = imap.Map(basemap=1, layout=Layout(height='600px'))
display(m)

# Creation of the bivariate legend
colorlist1 = ['#f3f3f3', '#eac5dd', '#e6a3d0']
colorlist2 = ['#f3f3f3', '#c2f1d5', '#8be2ae']

svg = interMap.bivariateLegend(vector,
                               ["[SIGLA] < 'FE'", "[SIGLA] >= 'FE' and [SIGLA] <= 'PU'", "[SIGLA] > 'PU'"],
                               ["[SHAPE_AREA] < 2500000000", "[SHAPE_AREA] >= 2500000000 and [SHAPE_AREA] <= 4500000000", 
                                "[SHAPE_AREA] > 4500000000 and [SHAPE_AREA] <= 7500000000", '[SHAPE_AREA] > 7500000000'],
                               colorlist1,
                               colorlist2,
                               title='Example of Bivariate Choropleth',
                               title1="Province initials",
                               names1=['< FE', 'in [FE,PU]', '> PU'],
                               title2="Province area",
                               names2=['Small', 'Medium', 'Large', 'XLarge'],
                               fontsize=24,
                               fontweight=500)

# Display of the vector layer on the map
p = vector.process()
m.clear()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

# Display the bivariate choropleth legend
display(HTML(svg))

### <b>Creation of a trivariate choropleth legend for a polygon vector layer</b>
See [Choropleth maps with tricolore](https://cran.r-project.org/web/packages/tricolore/vignettes/choropleth_maps_with_tricolore.html) for the idea. The function creates a legend for vector layer v based on three attributes of the layer and returns a string containing the SVG representation of the legend in the form of a triangle (that can be displayed using display(HTML(svgstring) call)

In [ ]:
from ipywidgets import widgets, Layout, HTML
from IPython.display import display

from jeodpp import inter, imap
from voilalibrary import interMap

# Load data
vector = inter.VectorLayer('wkt')
vector.fileLink('/eos/jeodpp/data/projects/JEODPP/view/mapnik/DEBY_2019_LandCover_LAU.shp','',4326)
vector = vector.parameter("identifyfield", "LAU_NAME AL_PERC PC_PERC PG_PERC")
vector = vector.parameter("identifyseparator", "</br>")

# Create and display a Map instance
m = imap.Map(basemap=60, layout=Layout(height='600px'))
display(m)

svg = interMap.trivariateLegendEx(vector,
                                  "AL_PERC",
                                  "PC_PERC",
                                  "PG_PERC",
                                  6,
                                  0.0,
                                  100.0,
                                  0.0,
                                  50.0,
                                  0.0,
                                  100.0,
                                  color1='#ffeb3b',
                                  color2='#b71c1c',
                                  color3='#43a047',
                                  color4='#ffffff00',
                                  title='Trivariate Choropleth',
                                  title1="Arable Land",
                                  title2="Perm. Crop",
                                  title3="Perm. Grassland",
                                  fontsize=20,
                                  fontweight=500,
                                  side=400,
                                  resizewidth='27vw',
                                  resizeheight='34vh',
                                  digits=0,
                                  maxticks=5,
                                  showarrows=True)

# Display of the vector layer on the map
p = vector.process()
m.clear()
m.addLayer(p.toLayer())
m.zoomToImageExtent(p)

inter.identifyPopup(m,p)

# Display the trivariate choropleth legend
display(HTML(svg))

### See https://jeodpp.jrc.ec.europa.eu/eu/dashboard/voila/render/Vaas/examples/vuetify/appExample3.ipynb